In [18]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from glob import glob as glob_module
import os

import librosa
import librosa.display

from python_speech_features import mfcc

import logging
import soundfile as sf
import sounddevice as sd

from joblib import delayed, Parallel

import json
import pickle
from pprint import pprint

In [19]:
logging.basicConfig(level=logging.INFO, filename="log.log", filemode="w")
logger = logging.getLogger(__name__)

handler = logging.FileHandler("dataprocessing_test2.log", mode="w")
formatter = logging.Formatter("%(name)s - %(message)s")
handler.setFormatter(formatter)

logger.addHandler(handler)

In [20]:
curr_notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(curr_notebook_dir,os.pardir))
train_path = r"data\raw\LibriSpeech\train-clean-100\LibriSpeech\train-clean-100\**\**\*.flac"
test_path = r"data\raw\LibriSpeech\test-clean\LibriSpeech\test-clean\**\**\*.flac"
dev_path = r"data\raw\LibriSpeech\dev-clean\LibriSpeech\dev-clean\**\**\*.flac"

train_path_whole = os.path.join(parent_dir,train_path)
test_path_whole = os.path.join(parent_dir,test_path)
dev_path_whole = os.path.join(parent_dir,dev_path)

In [21]:
data_files_train = glob_module(train_path_whole, recursive=True)
data_files_dev = glob_module(dev_path_whole, recursive=True)
data_files_test = glob_module(test_path_whole, recursive=True)

In [22]:
def corpus(file):
    file_name = os.path.basename(file)
    file_name_abs = file_name.split(".")
    cur_dir = os.path.dirname(file)
    transcript_file = glob_module(os.path.join(cur_dir, "*.txt"))
    with open(transcript_file[0],"r") as f:
        lines = f.readlines()
    first_words_list = [(line.split()[0],i) for i,line in enumerate(lines)]
    for elements in first_words_list:
        if elements[0]==file_name_abs[0]:
            line = lines[elements[1]]
            line = line.split()
            transcript = " ".join(line[1:])
            break
    return {"name": file_name_abs[0], "file": file, "transcript": transcript}

In [23]:
ordered_transcripts_train = []
for file in data_files_train:
    transcript = corpus(file)
    ordered_transcripts_train.append(transcript)

In [24]:
ordered_transcripts_test = []
for file in data_files_test:
    transcript = corpus(file)
    ordered_transcripts_test.append(transcript)

In [25]:
ordered_transcripts_dev = []
for file in data_files_dev:
    transcript = corpus(file)
    ordered_transcripts_dev.append(transcript)

In [26]:
corpus_path = r"data\processed"
curr_notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(curr_notebook_dir,os.pardir))
corpus_path_whole = os.path.join(parent_dir,corpus_path)

In [27]:
train_corpus_file = os.path.join(corpus_path_whole, "train_corpus_new.json")
with open(train_corpus_file, "w") as f:
    json.dump(ordered_transcripts_train, f)

In [28]:
test_corpus_file = os.path.join(corpus_path_whole, "test_corpus_new.json")
with open(test_corpus_file, "w") as f:
    json.dump(ordered_transcripts_test, f)

In [29]:
dev_corpus_file = os.path.join(corpus_path_whole, "dev_corpus_new.json")
with open(dev_corpus_file, "w") as f:
    json.dump(ordered_transcripts_dev, f)